# Use the IBM watsonx.governance metrics toolkit to evaluate watsonx.ai

The IBM watsonx.governance metrics toolkit lets you evaluate the output of a Large Language Model (LLM) against multiple task types: Text Summarization, Content Generation, Question Answering, Text Classification, Entity Extraction, and Retrieval-Augmented Generation (RAG).

This notebook will demonstrate how to evaluate output from a Text Summarization prompt run against an IBM watsonx.ai LLM. Evaluated metrics are then published to IBM OpenPages.

## Learning goals

The learning goals of this notebook are:

-  Create your prompt for testing against the watsonx.ai model. 
-  Configure metrics for evaluation.
-  Run the metrics against your prompt data.
-  Print and review the metrics returned by the IBM watsonx.governance metrics toolkit. 
-  Publish the computed metrics to IBM OpenPages.

## Table of Contents

This notebook contains the following parts:

1.	[Install the necessary packages](#packages)
2.  [Provision services and configure credentials](#credentials)
3.	[Evaluate Text Summarization output from the watsonx.ai model](#summarization)
4.	[Publish computed metrics to an OpenPages foundation model](#openpages)
5.  [Navigate to the OpenPages UI to verify your metrics](#verify)
6.	[Summary](#summary)

<a id="packages"></a>
## Step 1 - Install the necessary packages 

In [ ]:
!pip install --upgrade ibm-watson-machine-learning   | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

**Note**: If you are running this notebook in a Jupyter environment in a Linux install, you must install the `yum install postgresql-devel` library by running the upgrade code below:

In [ ]:
!pip install --upgrade ibm-metrics-plugin --no-cache | tail -n 1

In [ ]:
!pip install --upgrade evaluate --no-cache | tail -n 1
!pip install --upgrade rouge_score --no-cache | tail -n 1
!pip install --upgrade textstat --no-cache | tail -n 1
!pip install --upgrade sacrebleu --no-cache | tail -n 1
!pip install --upgrade sacremoses --no-cache | tail -n 1
!pip install --upgrade datasets==2.10.0 --no-cache | tail -n 1

In [ ]:
!pip install sqlalchemy==1.4.47
!pip install datasets==2.10.0

In [ ]:
import warnings
warnings.filterwarnings('ignore')

<a id="credentials"></a>
## Step 2 - Provision services and configure credentials

### Provision an instance of IBM Watson OpenScale

If you have not already done so, provision an instance of IBM Watson OpenScale using the [OpenScale link in the Cloud catalog](https://cloud.ibm.com/catalog/services/watson-openscale).

### Generate an API key

You can generate a Cloud API key with IBM Cloud console or with IBM Cloud command line interface.

To generate an API key by using IBM Cloud console:

1. Go to the [**Users** section of the IBM Cloud console](https://cloud.ibm.com/iam#/users).
1. Click your name, then scroll down to the **API Keys** section.
1. Click **Create an IBM Cloud API key**.
1. Give your key a name and click **Create**.
1. Copy the created key - you will need to paste this key into the `CLOUD_API_KEY` variable in the "Configure your credentials" section below.

To create an API key using the IBM Cloud [command line interface](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli):

1. From the command line interface, type the following:

    `bx login --sso`

    `bx iam api-key-create 'my_key'`

1. Copy the created key - you will need to paste this key into the `CLOUD_API_KEY` variable in the "Configure your credentials" section below.

### Configure your watsonx.ai credentials

In [ ]:
GEN_API_KEY = "<Your API Key>"
CLOUD_API_KEY = GEN_API_KEY
api_endpoint = "<IBM watsonx.ai model inferencing endpoint>"
project_id = "<Your IBM watsonx.ai project id>"
endpoint_url = "https://us-south.ml.cloud.ibm.com"

If you are running your notebook on a CPD cluster, uncomment and run the following code:

In [ ]:
# use_cpd = True
# WOS_CREDENTIALS = {
#     "url": "xxxxx",
#     "username": "xxxxx",
#     "api_key": "xxxxx"
# }

# GEN_API_KEY = WOS_CREDENTIALS["api_key"]

# api_endpoint = WOS_CREDENTIALS["url"]
# project_id = "<Your project id>"
# endpoint_url = WOS_CREDENTIALS["url"]

### Configure your OpenPages connection details

In [ ]:
OP_URL = "<OpenPages URL>"
OP_USERNAME = "OpenPages Username"
OP_PASSWORD = "OpenPages User Password"
model_name = 'OpenPages FM Model Id to which metrics needs to be published'

<a id="summarization"></a>
## Step 3 - Evaluate Text Summarization output from the watsonx.ai model

### Download a dataset containing prompt input data for model inferencing and reference data for model output evaluation

The downloaded `.csv` file contains: input, a generated summary, and two reference summaries each for 50 sample prompts. Values are then further converted to input, output and reference panda data frames.

In [ ]:
!rm -fr llm_content.csv
!wget "https://raw.githubusercontent.com/ravichamarthy/custom_metrics/main/llm_content.csv"

In [ ]:
import pandas as pd
import numpy as np
llm_data_all = pd.read_csv("llm_content.csv")
llm_data_all.head()

In [ ]:
llm_data = llm_data_all.head(10)
llm_data.head()

### Create your prompt for testing against the watsonx.ai model

Use the following code to build the prompt.

In [ ]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes, DecodingMethods

In [ ]:
generate_params = {
    GenParams.MAX_NEW_TOKENS: 75,
    GenParams.MIN_NEW_TOKENS: 10,
    GenParams.TEMPERATURE: 0.0
}

model = Model(
    model_id=ModelTypes.FLAN_UL2,
    params=generate_params,
    credentials={
        "apikey": GEN_API_KEY,
        "url": endpoint_url
    },
    project_id=project_id
)

### Run the prompt evaluation

Evaluate the prompt that you built against the IBM watsonx.ai model.

In [ ]:
def get_prompt(text):
    prompt = f"""Please provide a summary of the following text with maximum of 20 words.
    
{text}
    
Summary:"""
    return prompt

In [ ]:
def get_completion(input_text):
    prompt_text = get_prompt(input_text)
    model_response = model.generate_text(prompt=prompt_text)
    return model_response

### Set the generated prompt summary with the summary from the watsonx.ai prompt evaluation

In [ ]:
llm_data['watsonx_ai_generated_summary'] = llm_data['input_text'].apply(get_completion)

In [ ]:
llm_data.head()

#### Sample generated output

In [ ]:
llm_data['watsonx_ai_generated_summary'][0]

### Authenticate with IBM watsonx.governance

In [ ]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,BearerTokenAuthenticator,CloudPakForDataAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

if use_cpd:
    authenticator = CloudPakForDataAuthenticator(
            url=WOS_CREDENTIALS['url'],
            username=WOS_CREDENTIALS['username'],
            password=WOS_CREDENTIALS['password'],
            disable_ssl_verification=True
        )
    
    client = APIClient(service_url=WOS_CREDENTIALS['url'],authenticator=authenticator)
    print(client.version)
else:
    authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
    client = APIClient(authenticator=authenticator)
    print(client.version)

### Import common evaluation metrics and metric groups

These are the metrics used to evaluate your prompt against the selected model, based on the prompt task type. For example, Summarization.

In [ ]:
from ibm_metrics_plugin.metrics.llm.utils.constants import LLMTextMetricGroup
from ibm_metrics_plugin.metrics.llm.utils.constants import LLMGenerationMetrics
from ibm_metrics_plugin.metrics.llm.utils.constants import LLMSummarizationMetrics
from ibm_metrics_plugin.metrics.llm.utils.constants import LLMQAMetrics
from ibm_metrics_plugin.metrics.llm.utils.constants import LLMClassificationMetrics
from ibm_metrics_plugin.metrics.llm.utils.constants import HAP_SCORE
from ibm_metrics_plugin.metrics.llm.utils.constants import PII_DETECTION

### Get the necessary data for evaluating the prompt template metrics

Metrics will be evaluated for the input, output, and reference summary text.

In [ ]:
df_input = llm_data[['input_text']].copy()
df_output = llm_data[['watsonx_ai_generated_summary']].copy()
df_reference = llm_data[['reference_summary_2']].copy()

### Configure metrics for evaluation

Select the metrics you want to evaluate; the below code cell contains 10 common Summarization metrics.

In [ ]:
metric_config = {   
    "configuration": {
        LLMTextMetricGroup.SUMMARIZATION.value: {
            LLMSummarizationMetrics.ROUGE_SCORE.value: {},
            LLMSummarizationMetrics.SARI.value: {},
            LLMSummarizationMetrics.METEOR.value: {},
            LLMSummarizationMetrics.NORMALIZED_RECALL.value: {},
            LLMSummarizationMetrics.NORMALIZED_PRECISION.value: {},
            LLMSummarizationMetrics.NORMALIZED_F1_SCORE.value: {},
            LLMSummarizationMetrics.COSINE_SIMILARITY.value: {},
            LLMSummarizationMetrics.JACCARD_SIMILARITY.value: {},
            LLMSummarizationMetrics.BLEU.value: {},
            LLMSummarizationMetrics.FLESCH.value: {}
        }
    }
}

### Summarization metrics evaluation

Run the metrics against your prompt data. 

In [ ]:
import json
result = client.llm_metrics.compute_metrics(metric_config,df_input,df_output,df_reference)

### Review metrics

In [ ]:
print(json.dumps(result,indent=2))

<a id="openpages"></a>
## Step 4 - Publish computed metrics to an OpenPages foundation model

### Construct a key/value dictionary of the metrics to be published to OpenPages 

In [ ]:
def get_metrics(result):
    metrics = {}
    metrics['rouge1'] = round(result['rouge_score']['rouge1']['metric_value'], 4)
    metrics['rouge2'] = round(result['rouge_score']['rouge2']['metric_value'], 4)
    metrics['rougeL'] = round(result['rouge_score']['rougeL']['metric_value'], 4)
    metrics['rougeLsum'] = round(result['rouge_score']['rougeLsum']['metric_value'], 4)
    metrics['meteor'] = round(result['meteor']['metric_value'], 4)
    metrics['sari'] = round(result['sari']['metric_value'], 4)
    metrics['cosine_similarity'] = round(result['cosine_similarity']['metric_value'], 4)
    metrics['jaccard_similarity'] = round(result['jaccard_similarity']['metric_value'], 4)
    return metrics

In [ ]:
metrics =  get_metrics(result)
metrics

In [ ]:
import requests
import base64
import json

### Get an authorization token for OpenPages

In [ ]:
def get_basic_auth_token(username, password):
    token = base64.b64encode(bytes('{0}:{1}'.format(username, password), 'utf-8')).decode("ascii")
    return token

### For a given model name, get the OpenPages model ID

In [24]:
def get_op_model_id(header, model_name):
    openpages_url = OP_URL.rstrip("/") + "/grc/api/query"
    # Prepare post payload
    get_id_payload = {
        "statement": "SELECT [Model].[Resource ID] FROM [Model] WHERE [Model].[Name] IN ('{0}')".format(model_name),
        "skipCount": 0
    }
    response = requests.post(openpages_url, json=get_id_payload, headers=header, verify=False).json()

    model_id = None
    if response is not None:
        if response.get("rows") is not None:
            rows = response.get("rows")
            if len(rows) != 0:
                fields = rows[0].get("fields")
                if fields is not None:
                    field = fields.get("field")
                    if len(field) != 0:
                        model_id = field[0]["value"]

    if model_id is None:
        print("Model ID not found.")
    else:
        print("Model ID fetched: " + model_id)
    return model_id

### For a given model ID, get the corresponding OpenPages metrics definitions map containing metric ID and its name

In [25]:
def get_op_model_metrics_definitions(header, model_id):
    openpages_url = OP_URL.rstrip("/") + "/grc/api/query"    
    get_metrics_payload = {
        "statement": "SELECT [Metric].[Resource ID], [Metric].[Name], [Metric].[Description] FROM [Model] JOIN [Metric] ON PARENT([Model]) WHERE [Model].[Resource ID]='{0}'".format(model_id),
        "skipCount": 0
    }
    print("Sending request to fetch all metrics associated with the model.")
    response = requests.post(openpages_url, json=get_metrics_payload, headers=header, verify=False).json()

    metrics_map = []

    if response is not None:
        if response.get("rows") is not None:
            rows = response.get("rows")
            if len(rows) != 0:
                for i in range(len(rows)):
                    fields = rows[i].get("fields")
                    if fields is not None:
                        field = fields.get("field")
                        metric_id_desc = {}
                        metric_id = None
                        metric_desc = None
                        for row in field:
                            if row.get('name') == 'Resource ID':
                                metric_id = row.get('value')
                            if row.get('name') == 'Description':
                                metric_desc = row.get('value')
                        metric_id_desc['metric_desc'] = metric_desc
                        metric_id_desc['metric_id'] = metric_id
                        metrics_map.append(metric_id_desc)
        print("Completed fetching, if any, all metrics associated with the model.")
        return metrics_map

### Construct the Metrics Object Payload for metrics creation

In [26]:
def get_metric_object_payload(primaryParentId, metric_name):
    metric_description = "watsonx.governance metric for '" + metric_name + "'"
    metric_object_payload = {
    	"name": metric_name,
    	"description": metric_description,
    	"typeDefinitionId": "Metric",
        "primaryParentId": primaryParentId,
    	"fields":
    	{
    		"field":
    		[
    			{
                    "name": "MRG-Metric:Data Source",
                    "dataType": "STRING_TYPE",
                    "value": "watsonx.governance"
                },
                {
            		"name": "MRG-Metric:Frequency",
            		"dataType": "ENUM_TYPE",
            		"enumValue": {
                		"name": "Multiple times a day"
                	}
            	}
    		]
    	}
    }
    return metric_object_payload

### Construct the Metrics Value Payload for creating and associating a metric value to a metric of a given model object

In [27]:
def get_metric_value_payload(primaryParentId, metric_name, metric_value):
    metric_description = "watsonx.governance metric for '" + metric_name + "'"
    metric_value_payload = {
        "typeDefinitionId": "MetricValue",
        "primaryParentId": primaryParentId,
        "description": metric_description,
        "fields": {
            "field": [
                {
                    "name": "MRG-Metric-Shared:Breach Status",
                    "dataType": "ENUM_TYPE",
                    "enumValue": {
                        "name": "Green"
                    }
                },
                {
                    "name": "MRG-Metric-Shared:Red Threshold",
                    "dataType": "FLOAT_TYPE",
                    "value": 0.5
                },
                {
                    "name": "MRG-MetricVal:Value",
                    "dataType": "FLOAT_TYPE",
                    "value": metric_value
                }
            ]
        }
    }
    return metric_value_payload

### Create a Metrics Object

In [28]:
def create_metrics_object(metric_object_payload):
    openpages_metric_object_creation_url = OP_URL + "/grc/api/contents"
    response = requests.post(openpages_metric_object_creation_url, json=metric_object_payload, headers=header, verify=False).json()
    metric_id = response['id']
    return metric_id

### Add Metric Value to the Metric Object

In [29]:
def add_metric_value_to_metric_object(metric_value_payload):
    openpages_metric_value_creation_url = OP_URL + "/grc/api/contents"
    response = requests.post(openpages_metric_value_creation_url, json=metric_value_payload, headers=header, verify=False).json()
    metric_value_id = response['id']
    return metric_value_id

### Check for the metric's existence in the metrics map

In [30]:
def get_existing_metric_id(metrics_map, metric_name):
    for item in metrics_map:
        if 'metric_desc' in item and metric_name in item['metric_desc']:
            return item['metric_id']
    return None

### Get your OpenScale authorization token

In [31]:
token = get_basic_auth_token(OP_USERNAME, OP_PASSWORD)

In [ ]:
header = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "Authorization": "Basic {0}".format(token)
    }

### Fetch the Model ID for a given OpenPages model name

In [ ]:
model_id = get_op_model_id(header, model_name)
model_id

### Publish the metrics to OpenPages

In [ ]:
### Fetch the existing, if any, OP Model Metrics for a given OP Model ID
metrics_map = get_op_model_metrics_definitions(header, model_id)
print(metrics_map)

print('\n')

# Iterate over the given metrics to be published..
for metric_name, metric_value in metrics.items():
    
    # check if the metric exists by the given name, and if, get its metric_id
    metric_id = get_existing_metric_id(metrics_map, metric_name)

    # if the metric does not exists, then create it
    if metric_id is None:
        print(metric_name + ': Metric Object does not exists, hence creating it..')

        # construct the metric object to be published
        metric_object_payload = get_metric_object_payload(model_id, metric_name)

        # now, create the metric object
        metric_id = create_metrics_object(metric_object_payload)

    # Add the metric value to metric object

    # construct the metric value object to be published
    metric_value_payload = get_metric_value_payload(metric_id, metric_name, metric_value)

    # create the metric value - basically add the metric value to the metric object
    metric_value_id = add_metric_value_to_metric_object(metric_value_payload)
    
    print(str(metric_name) + ': Metric Object ID: ' + str(metric_id) + ', Metric Value Object ID: '+ str(metric_value_id) + '\n')

<a id="verify"></a>
## Step 5 - Navigate to the OpenPages UI to verify your metrics

To locate your metrics, first navigate to your model:
 1. From the **Menu** on the left, click on **Inventory**, and go to **Models**. 
 2. Choose your model from the list. 
 3. Go to the **Admin** tab
 4. To find your metrics, scroll down to **Associations**. They are under **Model Metrics**. 
 5. To view the metrics in a seperate tab, go to the icon in the top right corner and click **Launch Grid page**.

<a id="summary"></a>
## Summary

Congratulations, you successfully completed this notebook! You learned how to evaluate output from a Text Summarization prompt run against an IBM watsonx.ai LLM and publish the computed metrics to IBM OpenPages.

### Author:

**Ravi Chamarthy**